# Introduction

**Overview:** Brief description of the problem, the dataset, and the main objectives of the project.

# Setup 

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Environment Variables 
**Note**: Setting environment variables is optional, but it is recommended if you store sensitive information (such as API keys or database credentials) in a `.env` file. Using environment variables helps keep such information secure and separate from your codebase.

In [ ]:
# Imports
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API key from .env 
api_key = os.getenv("API_KEY")

# Get SQL database credentials from .env
sql_username = os.getenv("SQL_USERNAME")
sql_password = os.getenv("SQL_PASSWORD")

# Data Loading

## csv 

In [ ]:
# Load data from a csv file into a Pandas DataFrame
df = pd.read_csv("your_csv_file_here.csv")

## MySQL

In [ ]:
# Imports 
from sqlalchemy import create_engine

# Database info
mysql_host = "localhost"  # Default hostname for a MySQL server running locally
mysql_port = 3306  # Default port for MySQL
mysql_database_name = "your_mysql_database_name_here"
mysql_table_name = "your_mysql_table_name_here"

# Create an SQLAlchemy engine for interacting with the MySQL database
engine = create_engine(f"mysql+mysqlconnector://{sql_username}:{sql_password}@{mysql_host}:{mysql_port}/{mysql_database_name}")

# Load data from MySQL database into a Pandas DataFrame
with engine.connect() as connection:
    df = pd.read_sql(f"SELECT * FROM {mysql_table_name}", con=connection)

# Data Quality Checks

In [ ]:
# Show DataFrame info
df.info()

In [ ]:
# Show top five rows
df.head()

# Data Preprocessing

## Handling Duplicates

Duplicates based on all columns:

In [ ]:
# Diagnose duplicates 
df.duplicated().value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates().copy()

Duplicates based on specific columns, e.g. the ID column or a combination of columns:

In [ ]:
# Diagnose duplicates
df.duplicated(subset=["column_1", "column_2", "column_3"]).value_counts()

In [ ]:
# Remove duplicates
df = df.drop_duplicates(subset=["column_1", "column_2", "column_3"]).copy()

## Data Type Conversion

In [ ]:
# Convert column from str to int
df["int_column"] = df["str_column"].astype("Int32")

In [ ]:
# Convert column from str to datetime
df["datetime_column"] = pd.to_datetime(df["str_column"])

## Feature Extraction

### Categorical Feature from String Column

In [ ]:
# Function to extract a category from a string   
def extract_category_from_string(string):
    # Map categories to their corresponding list of keywords
    category_keywords_map = {
        "Category 1": ["Keyword 1", "Keyword 2", "Keyword 3"],
        "Category 2": ["Keyword 4", "Keyword 5", "Keyword 6"],
        "Category 3": ["Keyword 7", "Keyword 8", "Keyword 9"]
    }

    # Loop through each category and its associated keywords 
    for category, keywords in category_keywords_map.items():
        # Check if any keyword is present in the string
        if any(keyword in string for keyword in keywords):
            return category  # Return the category corresponding to the keyword
    return np.nan  # Return a missing value if no keyword matches

# Apply function on an existing string column to create a new categorical feature column
df["categorical_feature"] = df["str_column"].apply(extract_category_from_string)

# Show category frequencies
print(df["categorical_feature"].value_counts())

### Numerical Feature from String Column

In [ ]:
# Imports
import re

# Function to extract the first number in a string 
def extract_number_from_string(string):
    first_number = re.search(r"\b-?\d+([.,]\d+)?\b", string)  # searches for first integer or float (positive or negative; decimal separator "." or ",")
    if first_number:
        return float(first_number.group().replace(",", "."))  # Replace "," with "." as decimal separator  
    else:
        return np.nan  # Return a missing value if no number in string

# Apply function on an existing string column to create a new numerical feature column
df["numerical_feature"] = df["str_column"].apply(extract_number_from_string)

# Show descriptive statistics of numerical feature
df["numerical_feature"].describe()

### Boolean Feature from String Column

In [ ]:
# List of keywords to determine if the feature is present or absent
keywords = ["keyword 1", "keyword 2", "keyword 3"]

# Extract boolean feature column: True if any keyword is found in the string column
df["boolean_feature"] = df["str_column"].apply(lambda x: any(keyword.lower() in x.lower() for keyword in keywords))

# Show frequencies of boolean feature
print(df["boolean_feature"].value_counts())

## Handling Missing Values

### Imputation

Imputation for numerical column:

In [ ]:
# Descriptive statistics of numerical column
df["numerical_column"].describe()

In [ ]:
# Impute missing values with the median
median = df["numerical_column"].median()
df["numerical_column"] = df["numerical_column"].fillna(median)

Imputation for categorical column:

In [ ]:
# Frequencies of categorical column
df["categorical_column"].value_counts()

In [ ]:
# Impute missing values with the mode 
mode = df["categorical_column"].mode()[0]
df["categorical_column"] = df["categorical_column"].fillna(mode)

### Deletion

In [ ]:
# Delete rows where any column has a missing value 
df.dropna(inplace=True)

In [ ]:
# Delete rows where either column_1 or column_2 has a missing value 
df.dropna(subset=["column_1", "column_2"], how="any", inplace=True)

# Exploratory Data Analysis (EDA)

## Column Types

In [ ]:
# Show columns and their data types
df.info()

In [ ]:
# Define numerical, categorical and boolean columns
numerical_columns = ["numerical_column_1", "numerical_column_2", "numerical_column_3"]
categorical_columns = ["categorical_column_1", "categorical_column_2", "categorical_column_3"]
boolean_columns = ["boolean_column_1", "boolean_column_2", "boolean_column_3"]

## Descriptive Statistics

In [ ]:
# Table with descriptive statistics of all numerical columns
df[numerical_columns].describe().transpose()

## Numerical Label

### Histogram

In [ ]:
# Create histogram  of numerical label
sns.histplot(df["numerical_label"])

# Add title and axes labels 
plt.title("Distribution of numerical_label")
plt.xlabel("numerical_label")
plt.ylabel("Frequency")

# Show the plot
plt.show()

### Correlations

In [ ]:
# Correlations between the numerical label and each numerical and boolean column
combined_columns = numerical_columns + boolean_columns
df[combined_columns].corr()["numerical_label"].sort_values(ascending=False)

### Scatterplots 
Scatterplot matrix that shows scatterplots between the numerical label and each numerical column.   
Example code for 9 scatterplots in a 3x3 matrix:

In [ ]:
# Set the figure size 
plt.figure(figsize=(12, 12))

# List of numerical columns excluding the label column
numerical_columns_without_label = [col for col in numerical_columns if col != "numerical_label"]

# Iterate over the numerical columns
for i, numerical_column in enumerate(numerical_columns_without_label):
    # Create a subplot in a 3x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(3, 3, i + 1)
    
    # Create a scatterplot between the current column and the numerical label
    sns.scatterplot(data=df, x=numerical_column, y="numerical_label")
    
    # Add title and axis labels 
    plt.title(f"numerical_label by {numerical_column}")
    plt.xlabel(f"{numerical_column}")
    plt.ylabel("numerical_label")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

### Numerical Label by Category

Descriptive statistics:

In [ ]:
# Descriptive statistics of numerical label by categorical column
label_by_category = df["numerical_label"].groupby(df["categorical_column"])
label_by_category.describe()

Bar plot:

In [ ]:
# Store the median of the numerical label by category
median_label_by_category = label_by_category.median()

# Bar plot of median by category
sns.barplot(x=median_label_by_category.index, y=median_label_by_category.values, palette="colorblind")

# Add title and axes labels
plt.title("Median numerical_label by categorical_column")
plt.xlabel("Category")
plt.ylabel("numerical_label")

# Show the plot
plt.show()

Bar plot matrix that shows bar plots between the numerical label and each categorical column.  
Example code for 5 bar plots in a 2x3 matrix:

In [ ]:
# Set the figure size to 12x6 inches
plt.figure(figsize=(12, 6))

# Iterate over the categorical columns
for i, categorical_column in enumerate(categorical_columns):
    # Create a subplot in a 2x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(2, 3, i + 1)
    
    # Create a bar plot of median numerical_label by the current categorical column
    ax = sns.barplot(data=df, x=categorical_column, y="numerical_label", estimator=np.median, ci=None)
    
    # Add title and axes labels
    plt.title(f"Median numerical_label by {categorical_column}")
    plt.xlabel("Category")
    plt.ylabel("numerical_label")

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

## Categorical Columns

### Frequencies

#### Single Column

Absolute and relative frequencies of a single categorical colum:

In [ ]:
# Calculate absolute and relative frequencies 
absolute_frequencies = df["categorical_column"].value_counts()
relative_frequencies = absolute_frequencies / absolute_frequencies.sum() * 100

# Show frequencies
print(f"Absolute frequencies:\n {absolute_frequencies}\n")
print(f"Relative frequencies:\n {relative_frequencies.round(1)}")

Bar plot of frequencies:

In [ ]:
# Bar plot
sns.barplot(x=absolute_frequencies.index, y=absolute_frequencies.values, palette="colorblind")

# Add title and axes labels 
plt.title("categorical_column")
plt.ylabel("Frequency")

# Rotate x-axis tick labels by 45 degrees
plt.xticks(rotation=45)

# Show the plot
plt.show()

#### Multiple Columns

Absolute and relative frequencies of multiple categorical colums:

In [ ]:
# Initialize dictionaries to store frequencies for multiple categorical columns
absolute_frequencies_dict = {}
relative_frequencies_dict = {}

# Iterate over the categorical columns
for categorical_column in categorical_columns:
    # Calculate absolute and relative frequencies 
    absolute_frequencies = df[categorical_column].value_counts()
    relative_frequencies = absolute_frequencies / absolute_frequencies.sum() * 100

    # Store frequencies
    absolute_frequencies_dict[categorical_column] = absolute_frequencies
    relative_frequencies_dict[categorical_column] = relative_frequencies

    # Show frequencies
    print(categorical_column.upper())
    print(f"Absolute frequencies:\n {absolute_frequencies}\n")
    print(f"Relative frequencies (%):\n {relative_frequencies.round(1)}")
    print("=" * 50 + "\n")

Bar plot matrix that shows frequencies for multiple categorical columns.  
Example code for 5 bar plots in a 2x3 matrix:

In [ ]:
# Set the figure size
plt.figure(figsize=(12, 6))

# Iterate over the categorical columns
for i, categorical_column in enumerate(categorical_columns):
    # Create a subplot in a 2x3 grid (current subplot i+1 because subplot indices start at 1)
    plt.subplot(2, 3, i + 1)
    
    # Calculate frequencies for the current column
    absolute_frequencies = df[categorical_column].value_counts()
    
    # Create bar plot for the current column
    sns.barplot(x=absolute_frequencies.index, y=absolute_frequencies.values, estimator=np.median, ci=None)
    
    # Add title and axes labels
    plt.title(categorical_column.title())
    plt.ylabel("Frequency")

    # Rotate x-axis tick labels by 45 degrees
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

## Correlations

Correlation matrix:

In [ ]:
# Combine numerical and boolean columns for correlation analysis
combined_columns = numerical_columns + boolean_columns

# Calculate the correlation matrix 
correlation_matrix = df[combined_columns].corr()

# Round correlations to 2 decimals
correlation_matrix = round(correlation_matrix, 2)

# Create an upper triangle mask (k=1 excludes the diagonal)
mask = np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)

# Set upper triangle to NaN to avoid redundant information
correlation_matrix[mask] = np.nan

# Show correlation matrix
correlation_matrix

Correlation heatmap:

In [ ]:
# Set the figure size
plt.figure(figsize=(8, 6))

# Create heatmap
sns.heatmap(
    correlation_matrix, 
    cmap="viridis",  # Color map choice
    annot=True,  # Show numbers
    linewidth=0.5  # Thin white lines between cells
)

# Add title
plt.title("Correlation Heatmap")

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

Save correlation heatmap as an image:

In [ ]:
# Imports
import os

# Create images folder if it doesn't exist
os.makedirs("images", exist_ok=True)

# Save the heatmap 
try:
    # Construct full file path
    image_path = os.path.join("images", "correlation_heatmap.png") 
    
    # Save the heatmap as a .png image 
    plt.savefig(image_path, bbox_inches="tight", dpi=300)
    print(f"Correlation heatmap saved successfully to {image_path}")

except Exception as e:
    print(f"Error saving correlation heatmap: {e}")

# Train-Validation-Test Split

# Model Training